In [32]:
:opt no-lint

# 27 Non-strictness
## 27.1 Laziness
Haskell technically has _non-strict_ —not lazy— evaluation, but the difference between lazy and non-strict is not practically relevant.
Most expressions are only reduced, or evaluated, when necessary. When the evaluation process begins, a thunk is created for each expression.
- don’t do more work than needed;
- don’t evaluate until necessary;
- don’t re-evaluate if you don’t have to

## 27.2 Observational bottom theory
A truly lazy language _memoizes_ all the results of all the computations: this tends to use unacceptably large amounts of memory.
The essence of non-strictness is that you can have an expression that evaluates to a value, even if contains bottom or infinite data.
## 27.3 Outside in, inside out
Strict languages evaluate *inside out*; non-strict languages like Haskell evaluate _outside in_.
## 27.4 What does the other way look like?
In strict languages, you cannot ordinarily bind a computation to a name without having already done all the work to construct it.
## 27.5 Can we make Haskell strict?
The `seq` function forces evaluation of the first argument if and when the second argument has to be evaluated.
Since evaluation in Haskell is demand driven, we can’t guarantee that something will ever be evaluated. Instead, we have to create links between nodes in the graph of expressions where forcing one expression will force yet another expression.
#### seq and weak head normal form
What `seq` does is evaluate your expression up to *weak head normal form*, which means it stops at the first data constructor or lambda.
#### Case matching also chains evaluation
This forcing behavior happens already without `seq`! For example, when you case or pattern match on something, you’re forcing the value you pattern match on.
#### Core dump
Instead of injecting bottoms, we can analyze non-strictness by dumping the GHC Core language.
### Exercises: Evaluate
Expand the following expressions in as much detail as possible. Then, work outside-in to see what the expressions evaluate to:

1. 

In [33]:
const 1 undefined

1

2. 

In [34]:
const undefined 1

: 

3. 

In [35]:
flip const undefined 1

1

4. 

In [36]:
flip const 1 undefined

: 

5. 

In [37]:
const undefined undefined

: 

6. 

In [38]:
foldr const 'z' ['a'..'e']

'a'

7. 

In [39]:
foldr (flip const) 'z' ['a'..'e']

'z'

## 27.6 Call by name, call by need
1. Call by value: argument expressions are evaluated before entering a function. The expressions that bindings reference are evaluated before the bindings are created. This is conventionally called strict. This is inside-out evaluation.
2. Call by name: expressions can be arguments to a function without having been evaluated, or, in some cases, without ever being evaluated. You can create bindings to expressions without evaluating them first. Non-strictness includes this evaluation strategy. This is outside-in.
3. Call by need: this is the same as call by name, but expressions are only evaluated once. This only happens some of the time in GHC Haskell, usually when an expression isn’t a lambda that takes arguments and also has a name. Results are typically shared within that name only in GHC Haskell. This is also non-strict and outside-in.

## 27.7 Non-strict evaluation changes what we can do
## 27.8 Thunk Life
A _thunk_ is used to reference suspended computations that might be performed or computed at a later point in your program. They are computations that are not yet evaluated up to weak head normal form.
#### Not all values get thunked
The GHCi command `sprint` shows when something is thunked.
For opportunistic strictness, GHC will not thunk (and thus delay) data constructors which are known to be constant and ,thus, justifies the safety of the optimization.
Data constructors are like functions when they’re unapplied and like constants once they are fully applied.
GHC will also stop opportunistically evaluating as soon as it hits a computation.
## 27.9 Sharing is caring
GHC turns sharing on and off (that is, it oscillates between call-by-need and call-by-name) based on what it thinks will produce faster code. It can do this at all without breaking your code is because it knows when your code does or does not perform I/O.
## Using trace to observe sharing
The _base_ library has a module named `Debug.Trace`, useful for observing sharing. It is a means of cheating the type system and inserting a `putStrLn` into your code without having `IO` in the type.
#### What promotes sharing
GHC may enabled sharing if more parts of an expression reference the same name.
#### What subverts or prevents sharing
For example, inlined expressions and a functions with explicit, named arguments and type class constraints (which decay into function arguments when the compiler simplifies the code). Implicit parameters are implemented similarly to type class constraints and have the same effect on sharing.
#### Why polymorphic values never seem to get forced
Type constraints are like function (e.g.: `Num a -> a`). Only when all the parameters are applied (hence, the type become concrete) it becomes a constant shareable value.
#### Preventing sharing on purpose
We may want to diable sharing, e.g.: to avoid a large amount of data in memory. A technique is to create a dummy lambda like `x' = \_ -> x` to be called as `x' ()`. This technique doesn't work for point-free style.
#### Forcing sharing
You can force sharing by giving your expression a name. This technique is adoped by `forever` implementation to avoid a memory leak while endless running monadic actions.
## 27.10 Refutable and irrefutable patterns
- irrefutable pattern: one that will never fail to match
- refutable pattern: one that may not match

An irrefutable function works for any inhabitant of the type. If an irrefutable pattern comes before a refutable pattern, the refutable pattern will never get invoked.
### Lazy patterns
Lazy patterns (identified by a tiled) are also irrefutable. They cannot discriminate cases of a sum, but they are useful to unpack products that might not get used.

In [40]:
strictPattern :: (a, b) -> String
strictPattern (a, b) = const "Cousin It" a
strictPattern undefined

: 

In [41]:
lazyPattern :: (a, b) -> String
lazyPattern ~(a, b) = const "Cousin It" a
lazyPattern undefined

: 

## 27.11 Bang patterns
Sometimes, we want to evaluate an argument to a function whether we use it or not. We can do this with `seq` or with _bang patterns_

In [42]:
{-# LANGUAGE BangPatterns #-}

banging :: Bool -> Int
banging !b = 1

banging undefined

: 

In [43]:
banging' :: Bool -> Int
banging' b = 1

banging' undefined

: 

### Bang patterns in data
When we evaluate the outer data constructor of a datatype, at times we’d also like to evaluate the contents to weak head normal form, just like we do with functions.

In [44]:
data Foo = Foo Int !Int

first (Foo x _) = x
second (Foo _ y) = y

second (Foo undefined 1)
first (Foo 1 undefined)
Exception: Prelude.undefined

: 

Sometimes it’s cheaper to just compute something than to construct a thunk and then evaluate it later.
Genearal rule: lazy in the spine, strict in the leaves.
## 27.12 Strict and StrictData
The `Strict` and `StrictData` extensions are an alternative way to get a strict semantic.

In [45]:
{-# LANGUAGE Strict #-}

blah x = 1

print (blah undefined)

: 

## 27.13 Adding strictness
Adding strictness in different places produces different effects when the evaluation contains a bottom.
## 27.14 Chapter exercises
### Strict list
Try messing around with the following list type and compare what
it does with the bang-patterned list variants we experimented with
earlier:


```haskell
data List a = Nil | Cons !a (List a) deriving (Show)

take' n _  | n <= 0 = Nil
take' _ Nil         = Nil
take' n (Cons x xs) = (Cons x (take' (n - 1) xs))

map' _ Nil         = Nil
map' f (Cons x xs) = (Cons (f x) (map' f xs))

repeat' x = xs 
  where xs = (Cons x xs)

print $ take' 10 $ map' (+1) (repeat' 1)
```

### What will sprint output?
Determine `sprint` command output
1. 

In [46]:
let x = 1

:sprint x

x = _

2. 

In [47]:
let x = ['1']

:sprint x

x = "1"

3. 

In [48]:
let x = [1]

:sprint x

x = _

4. 

In [49]:
let x = 1 :: Int

:sprint x

x = 1

5. 

In [50]:
let f = \x -> x
let x = f 1

:sprint x

x = _

6. 

In [51]:
let f :: Int -> Int; f = \x -> x
let x = f 1

:sprint x

x = 1

#### Will printing this expression result in bottom?
1. 

In [52]:
snd (undefined, 1)

1

2. 

In [53]:
let x = undefined
let y = x `seq` 1 in snd (x, y)

:sprint x

: 

3. 

In [54]:
length $ [1..5] ++ undefined

: 

4. 

In [55]:
length $ [1..5] ++ [undefined]

6

5. 

In [56]:
const 1 undefined

1

6. 

In [57]:
const 1 (undefined `seq` 1)

1

7. 

In [58]:
const undefined 1

: 

### Make the expression bottom
Using only bang patterns or `seq`, make the code bottom out when executed:

In [59]:
{-# LANGUAGE BangPatterns #-}

x = undefined
y = "blah"

main = do
  print (snd (x, y))

main

"blah"

In [60]:
main' = do
  print $ x `seq` (snd (x, y))
  
main'

: 

In [61]:
main'' = do
  print $ go (x, y)
  where
    go (!x, y) = snd (x, y)

main''

: 